# Modelo Santi

In [36]:
import pandas as pd
import numpy as np
import datetime
from sklearn.model_selection import train_test_split

In [37]:
def oneHotEncoding(df, columna):
    one_hot = pd.get_dummies(df[columna])
    df.drop(columna, axis=1, inplace=True)
    df = pd.concat([df, one_hot], axis=1)
    return df

# Cargo CSV

In [38]:
test_final = pd.read_csv('data/test_final_100k.csv')
test_final = test_final.drop(columns='id')

In [39]:
postulantes_educacion = pd.read_csv('data_preprocesada/fiuba_1_postulantes_educacion.csv')
postulantes_educacion = postulantes_educacion.drop(columns='Unnamed: 0')
postulantes_genero_y_edad = pd.read_csv('data_preprocesada/fiuba_2_postulantes_genero_y_edad.csv')
postulantes_genero_y_edad = postulantes_genero_y_edad.drop(columns='Unnamed: 0')
vistas = pd.read_csv('data_preprocesada/fiuba_3_vistas.csv')
vistas = vistas.drop(columns='Unnamed: 0')
postulaciones = pd.read_csv('data_preprocesada/fiuba_4_postulaciones.csv')
postulaciones = postulaciones.drop(columns='Unnamed: 0')
avisos_online = pd.read_csv('data_preprocesada/fiuba_5_avisos_online.csv')
avisos_online = avisos_online.drop(columns='Unnamed: 0')
avisos_detalle = pd.read_csv('data_preprocesada/fiuba_6_avisos_detalle.csv')
avisos_detalle = avisos_detalle.drop(columns='Unnamed: 0')

# Funciones

In [40]:
# Fijarse de balancear el sample con 50% de postulados y no postulados
# Recibe las series de idpostulante e idaviso
def generador_de_sample_serie(idaviso, idpostulante, n_sample, ciclos):
    idpostulante = pd.DataFrame(idpostulante)
    idaviso = pd.DataFrame(idaviso)
    idpostulante['tem'] = 1
    idaviso['tem'] = 1
    sample_append = pd.DataFrame(columns=[idpostulante.columns[0],'tem',idaviso.columns[0]])
    for i in range(0,ciclos):
        sample_idpostulante = idpostulante.sample(n=n_sample,random_state=i)
        sample_idaviso = idaviso.sample(n=n_sample,random_state=i)
        sample_merge = sample_idaviso.merge(sample_idpostulante)
        sample_append = sample_append.append(sample_merge)
    sample_append = sample_append.drop(columns='tem')
    sample_append = sample_append.reset_index(drop=True)
    return sample_append

In [41]:
def borrar_tuplas_test_del_sample(sample,test_final):
    df_return = sample.merge(test_final,how='left',on=['idaviso','idpostulante'],indicator=True)
    df_return = df_return[df_return['_merge'] == 'left_only']
    df_return = df_return.drop(columns='_merge')
    return df_return

In [42]:
def genero_edad(tupla_idaviso_idpostulante,postulantes_genero_y_edad):
    df_return = tupla_idaviso_idpostulante.merge(postulantes_genero_y_edad,on='idpostulante',how='left')
    #df_return = df_return.drop(columns='fechanacimiento')
    return df_return

In [43]:
def educacion(tupla_idaviso_idpostulante):
    df_return = tupla_idaviso_idpostulante.merge(postulantes_educacion,on='idpostulante',how='left')
    return df_return

In [44]:
def area_de_trabajo(tupla_a_predecir):
    df_return = tupla_a_predecir.merge(avisos_detalle,on='idaviso',how='left')
    return df_return

In [45]:
def avisos_visitados(tupla_idaviso_idpostulante):
    df_return = tupla_idaviso_idpostulante.merge(vistas,on=['idpostulante','idaviso'],how='left')
    return df_return

In [46]:
def avisos_online_mark(tupla_idaviso_idpostulante):
    df_return = tupla_idaviso_idpostulante.merge(avisos_online,on=['idaviso'],how='left')
    return df_return

# ML

In [12]:
idpostulante = test_final['idpostulante'].drop_duplicates()

In [13]:
idaviso = test_final['idaviso'].drop_duplicates()

In [14]:
sample = generador_de_sample_serie(idaviso,idpostulante,100,200)
del idpostulante
del idaviso

In [15]:
sample = borrar_tuplas_test_del_sample(sample,postulaciones)
sample['y'] = 0
postulaciones['y'] = 1
sample = sample.append(postulaciones.sample(n=2000000,random_state=0))
sample.shape

(3998061, 3)

In [16]:
sample = sample.drop_duplicates()

In [17]:
sample = genero_edad(sample,postulantes_genero_y_edad)
sample['sexo'] = sample['sexo'].fillna('NO_DECLARA')
sample['edad'] = sample['edad'].astype('int8')
sample = oneHotEncoding(sample,'sexo')

In [18]:
sample = educacion(sample)
sample = sample.fillna(0)
#sample.iloc[:,7:] = sample.iloc[:,7:].astype('int8')

In [19]:
sample = area_de_trabajo(sample)
sample = sample.fillna(0)
#sample.iloc[:,14:] = sample.iloc[:,14:].astype('int8')

In [20]:
sample = avisos_visitados(sample)
sample = sample.fillna(0)
#sample.loc[:,'visto'] = sample.loc[:,'visto'].astype('int8')

In [21]:
sample = avisos_online_mark(sample)
sample = sample.fillna(0)
#sample.loc[:,'online'] = sample.loc[:,'online'].astype('int8')

In [22]:
del postulantes_educacion
del postulantes_genero_y_edad
del vistas
del postulaciones
del avisos_online
del avisos_detalle

In [23]:
sample = sample.drop(columns=['idaviso','idpostulante'])

In [24]:
sample.head()

,y,edad,tasa_x,FEM,MASC,NO_DECLARA,Doctorado,Master,Otro,Posgrado,...,Secretaria de Tráfico,Seguridad e Higiene,Seguros,Tecnologia/Sistemas,Tecnologia/Sistemas de producción,Ventas,Ventas económico-financiero,tasa_y,visto,online
0,0,30,0.0,0,1,0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,1,0.325,0.0,1.0
1,0,52,0.0,0,1,0,0.0,0.0,0.0,3.0,...,0,0,0,0,0,0,1,0.325,0.0,1.0
2,0,25,0.0,1,0,0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,1,0.325,0.0,1.0
3,0,42,0.4,0,1,0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,1,0.325,0.0,1.0
4,0,47,0.0,0,1,0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,1,0.325,0.0,1.0


In [25]:
# Divido el set
sample, X_test, sample_y_train, y_test = train_test_split(sample.drop(columns='y'),sample['y'],random_state=0,test_size=0.01)

# Perceptron

In [26]:
from sklearn.metrics import roc_auc_score

In [27]:
from sklearn.linear_model import Perceptron

In [28]:
model = Perceptron(penalty=None,n_jobs=-1)
model.fit(sample,sample_y_train)
predict = model.predict(X_test)

/usr/local/lib/python2.7/dist-packages/sklearn/linear_model/stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.perceptron.Perceptron'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)


In [29]:
roc_auc_score(y_test,predict)

0.7003680054619623

# KNN

In [ ]:
#from sklearn.neighbors import KNeighborsClassifier

In [ ]:
#model = KNeighborsClassifier(n_neighbors=3)
#model.fit(X_train,y_train)
#predict = model.predict(X_test)

In [ ]:
#roc_auc_score(y_test,predict)

# Random Forest

In [30]:
from sklearn.ensemble import RandomForestClassifier

In [31]:
model = RandomForestClassifier(n_jobs=-1)
model.fit(sample, sample_y_train)
predict = model.predict(X_test)

In [32]:
roc_auc_score(y_test,predict)

0.8965126379976642

# Gradient Boosting Classifier

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier

In [ ]:
model = GradientBoostingClassifier()
model.fit(sample, sample_y_train)
predict = model.predict(X_test)

In [ ]:
roc_auc_score(y_test,predict)

# ABB

In [33]:
from sklearn.tree import DecisionTreeClassifier

In [34]:
model = DecisionTreeClassifier()
model.fit(sample, sample_y_train)
predict = model.predict(X_test)

In [35]:
roc_auc_score(y_test,predict)

0.901087324028174

# Bayes

In [ ]:
from sklearn.naive_bayes import GaussianNB

In [ ]:
model = GaussianNB()
model.fit(sample, sample_y_train)
predict = model.predict(X_test)

In [ ]:
roc_auc_score(y_test,predict)

In [ ]:
from sklearn.naive_bayes import MultinomialNB

In [ ]:
model = MultinomialNB()
model.fit(sample, sample_y_train)
predict = model.predict(X_test)

In [ ]:
roc_auc_score(y_test,predict)

# Test Final

In [47]:
test_final = genero_edad(test_final,postulantes_genero_y_edad)
test_final['sexo'] = test_final['sexo'].fillna('NO_DECLARA')
#test_final['edad'] = test_final['edad'].astype('int8')
#test_final = oneHotEncoding(test_final,'sexo')

In [48]:
test_final = educacion(test_final)
test_final = test_final.fillna(0)
#test_final.iloc[:,6:] = test_final.iloc[:,6:].astype('int8')

In [49]:
test_final = area_de_trabajo(test_final)
test_final = test_final.fillna(0)
#test_final.iloc[:,13:] = test_final.iloc[:,13:].astype('int8')

In [50]:
test_final = avisos_visitados(test_final)
test_final = test_final.fillna(0)
#test_final.loc[:,'visto'] = test_final.loc[:,'visto'].astype('int8')

In [51]:
test_final = avisos_online_mark(test_final)
test_final = test_final.fillna(0)
#sample.loc[:,'online'] = sample.loc[:,'online'].astype('int8')

In [52]:
test_final = test_final.drop(columns=['idaviso','idpostulante'])

In [53]:
test_final.head()

,sexo,edad,tasa_x,Doctorado,Master,Otro,Posgrado,Secundario,Terciario/Técnico,Universitario,...,Secretaria de Tráfico,Seguridad e Higiene,Seguros,Tecnologia/Sistemas,Tecnologia/Sistemas de producción,Ventas,Ventas económico-financiero,tasa_y,visto,online
0,FEM,42,0.000000,3.0,0.0,0.0,0.0,0.0,0.0,3.0,...,0,0,0,0,0,1,0,0.0,0.0,0.0
1,MASC,31,0.000000,0.0,0.0,3.0,0.0,0.0,3.0,0.0,...,0,0,0,0,0,1,0,0.0,0.0,0.0
2,FEM,36,0.500000,0.0,0.0,0.0,0.0,0.0,0.0,3.0,...,0,0,0,0,0,1,0,0.0,0.0,0.0
3,MASC,69,0.000000,0.0,0.0,0.0,3.0,0.0,0.0,3.0,...,0,0,0,1,0,0,0,0.0,0.0,0.0
4,FEM,32,0.333333,0.0,0.0,0.0,0.0,0.0,2.0,2.0,...,0,0,0,1,0,0,0,0.0,0.0,0.0


In [36]:
predict = model.predict(test_final)

In [37]:
predict = pd.DataFrame(predict,columns=['sepostulo'])

In [38]:
predict.to_csv('prediccion')